In [32]:
import re
import itertools
import numpy as np
import pandas as pd

from copy import copy, deepcopy
import warnings
warnings.filterwarnings("ignore")

In [33]:
df = pd.read_csv("../data/raw/raw.csv", nrows=30000)
df.head(5)

,Номер события,Номер животного,Ушная бирка животного,Пол,Номер лактации,Результат отела,Легкость отела,Дата рождения,Дней в сухостое предыдущей лактации,Дней стельности при событии,Номер группы животного,Предыдущий номер группы животного,Событие,Дни доения при событии,Дата события,Примечание события
0,0,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,598621616,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,530073354,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


# Предобработка

In [34]:
col_mapper = {
    'Номер события'                      : 'event_id',
    'Пол'                                : 'sex',
    'Номер животного'                    : 'id',
    'Номер лактации'                     : 'lactation_number',
    'Результат отела'                    : 'birth_result',
    'Легкость отела'                     : 'birth_difficult',
    'Дата рождения'                      : 'birthday',
    'Дней в сухостое предыдущей лактации': 'days_no_milking',
    'Дней стельности при событии'        : 'days_pregnant',
    'Номер группы животного'             : 'group_id',
    'Предыдущий номер группы животного'  : 'prev_group_id',
    'Событие'                            : 'event',
    'Дни доения при событии'             : 'days_milking',
    'Дата события'                       : 'date',
    'Примечание события'                 : 'remark'
}

df.drop([col for col in df.columns if col not in col_mapper], axis=1, inplace=True)
df.columns = [col_mapper[col] for col in df.columns]
df.head()

,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark
0,0,146,F,6,FA,1,29.03.2010,62,277,18,18,РАСЧКОП,336,28.03.2019,КАН
1,1,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,КОГЛАВАК
2,2,146,F,6,FA,1,29.03.2010,62,277,18,18,ВАКЦИН,349,10.04.2019,ЛЕПТО
3,3,291,F,6,MA,1,30.07.2009,68,0,23,23,ПЕРЕВОД,448,08.04.2019,F001T023
4,4,291,F,6,MA,1,30.07.2009,68,0,23,23,ПРОДАНА,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8


## Проверка на пустые значения

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   event_id          30000 non-null  int64 
 1   id                30000 non-null  int64 
 2   sex               30000 non-null  object
 3   lactation_number  30000 non-null  int64 
 4   birth_result      21779 non-null  object
 5   birth_difficult   30000 non-null  int64 
 6   birthday          30000 non-null  object
 7   days_no_milking   30000 non-null  int64 
 8   days_pregnant     30000 non-null  int64 
 9   group_id          30000 non-null  int64 
 10  prev_group_id     30000 non-null  int64 
 11  event             30000 non-null  object
 12  days_milking      30000 non-null  int64 
 13  date              30000 non-null  object
 14  remark            30000 non-null  object
dtypes: int64(9), object(6)
memory usage: 3.4+ MB


В childs ставится NaN если ещё не было ребенка или животное мужского пола.
В remark стоит '-' если примечание не требуется

## Обработка событий

Привидем поле event к нормальному виду:

In [36]:
EventSynonims = {
    "ABORT"   : ["ABORT", "АБОРТ"],  
    "TOSCM"   : ["TOSCM", "НА_СХЕМУ"], 
    "NULSCM"  : ["NULSCM","СО_СХЕМЫ"],
    "DNB"     : ["DNB", "НЕОСЕМ"],
    "BRED"    : ["BRED", "ОСЕМЕН"], 
    "FRESH"   : ["FRESH", "ОТЕЛ"],  
    "PREG"    : ["PREG", "СТЕЛН"], 
    "PREGBEF" : ["PREGBEF", "СТЕЛНДО"],
    "DRY"     : ["DRY", "СУХОСТ"],
    "DRY2"    : ["DRY2", "СУХ2"],
    "OPEN"    : ["OPEN", "ЯЛОВАЯ"],
    #-------
    "WEIGHT"  : ["WEIGHT", "ВЕС"],
    "DEAD"    : ["DEAD", "ПАЛА"],
    "MOVE"    : ["MOVE", "ПЕРЕВОД"],
    "SOLD"    : ["SOLD", "ПРОДАНА"],
    #-------
    "VAC"     : ["VAC", "ВАКЦИН"],
    "VACVIR"  : ["VACVIR", "ВАКВИРУС"],
    "FOOTRIM" : ["FOOTRIM", "РАСЧКОП"],
    "HEALTH"  : ["HEALTH", "WELL", "ЗДОРОВА"],
    "BROKE"   : ["BROKE", "ДЕФЕКТ"],
    "POT"     : ["POT", "ПРОФОТ"],
    "ILLMISC" : ["ILLMISC", "БОЛЕЗНЬ"],
    "LAME"    : ["LAME", "ХРОМОТА"],
    "KETOS"   : ["KETOS", "КЕТОЗ"],
    "MAST"    : ["MAST", "МАСТИТ"],
    "METR"    : ["METR", "МЕТРИТ"],
    "PARES"   : ["PARES", "ПАРЕЗ"],
    "RP"      : ["RP", "ПОСЛЕД"],
}

EventName = dict()
for event in EventSynonims.keys():
    for label in EventSynonims[event]:
        EventName[label] = event
EventTypes = EventSynonims.keys()

Унифицируем все события к английскому:

In [37]:
df['event'] = df['event'].apply(lambda label: EventName[label])
df.head(10)

,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,prev_group_id,event,days_milking,date,remark
0,0,146,F,6,FA,1,29.03.2010,62,277,18,18,FOOTRIM,336,28.03.2019,КАН
1,1,146,F,6,FA,1,29.03.2010,62,277,18,18,VAC,349,10.04.2019,КОГЛАВАК
2,2,146,F,6,FA,1,29.03.2010,62,277,18,18,VAC,349,10.04.2019,ЛЕПТО
3,3,291,F,6,MA,1,30.07.2009,68,0,23,23,MOVE,448,08.04.2019,F001T023
4,4,291,F,6,MA,1,30.07.2009,68,0,23,23,SOLD,458,18.04.2019,ПРОДАНА ; Прочее ; УВЗ8
5,5,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,213,15.03.2019,МВ
6,6,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,215,17.03.2019,ТМ6_34
7,7,321,F,6,FA,3,11.09.2009,62,291,18,18,MOVE,216,18.03.2019,F006T003
8,8,321,F,6,FA,3,11.09.2009,62,291,18,18,MAST,221,23.03.2019,КМ4_134
9,9,321,F,6,FA,3,11.09.2009,62,291,18,18,HEALTH,230,01.04.2019,МАСТИТ


Посмотрим какие есть примечания

In [38]:
def remarks(event):
    return df[df['event'] == event]['remark'].unique().tolist()
    
for event in EventTypes:
    print(event + ':\n', remarks(event))
    print('---------------------------------------------------------------------------')

ABORT:
 ['189 ДНИ', '146 ДНИ', '205 ДНИ', '94 ДНИ', '95 ДНИ', '217 ДНИ', '93 ДНИ', '167 ДНИ', '182 ДНИ', '141 ДНИ', '184 ДНИ', '51 ДНИ', '208 ДНИ', '58 ДНИ', '69 ДНИ', '155 ДНИ', '46 ДНИ', '206 ДНИ', '65 ДНИ', '200 ДНИ', '90 ДНИ', '33 ДНИ', '194 ДНИ', '56 ДНИ', '68 ДНИ', '67 ДНИ', '96 ДНИ', '59 ДНИ', '89 ДНИ', '91 ДНИ', '48 ДНИ', '55 ДНИ', '188 ДНИ', '181 ДНИ', '136 ДНИ', '214 ДНИ', '196 ДНИ', '170 ДНИ', '61 ДНИ', '104 ДНИ', '120 ДНИ', '101 ДНИ', '62 ДНИ', '76 ДНИ', '84 ДНИ', '85 ДНИ', '219 ДНИ', '229 ДНИ', '192 ДНИ', '78 ДНИ', '39 ДНИ']
---------------------------------------------------------------------------
TOSCM:
 []
---------------------------------------------------------------------------
NULSCM:
 []
---------------------------------------------------------------------------
DNB:
 ['ЗООТЕХ', 'НОГИ', 'ВЫМЯ', 'СУСТАВ', 'ГИНЕКОЛ', 'АТП', 'AM', 'АМ', 'СОКОЛОВ', 'АБС', 'МАТКА']
---------------------------------------------------------------------------
BRED:
 ['1H11917', '1H11673',

**Механика парсинга**.


In [39]:
def splitter(s):
    return s.split(';')[1].strip().rstrip() if len(s.split(';')) > 1 else 'OTHER'


df['remark'] = df['remark'].apply(lambda x: x.replace('?', '1'))
df['remark'] = df.apply(lambda row: splitter(row.remark) if (row.event == 'SOLD' or row.event == 'DEAD') else row.remark, axis=1)

In [44]:
import itertools

def rule_name(s):
    symbols = (u'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ0123456789_', u'ABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA0123456789_')
    trans = {ord(a):ord(b) for a, b in zip(*symbols)}
    return ''.join(list(map(lambda val: val.translate(trans), s)))

class ParsingRule:
    def __init__(self, match, parser, name = ''):
        self.name  = rule_name(name)
        self.parse = parser
        self.match = match
    
    def __copy__(self):
        return ParsingRule(self.match, self.parse, copy(self.name))
   
    def copy(self):
        return ParsingRule(self.match, self.parse, copy(self.name))
    
    @staticmethod
    def match_in(match_list, parse, name=''):
        return ParsingRule(lambda x: x in match_list, parse, name)
    
    @staticmethod
    def match_str(match_str, parse, name=''):
        return ParsingRule(lambda x: x == match_str, parse, name)
    
    @staticmethod
    def match_reg(reg, parse, name=''):
        return ParsingRule(lambda x: reg.match(x), parse, name)


def satisfy(s, rules):
    return any([r.match(s) for r in rules])
    
def uniq(names):
    return [ParsingRule.match_str(s, lambda x: 1, rule_name(s)) for s in names]

def combi(rules, names):
    return rules + [ParsingRule.match_str(s, lambda x: 1, rule_name(s)) for s in names if not satisfy(s, rules)]

def comp(rules, names):
    return rules + [ParsingRule.match_in([s for s in names if not satisfy(s, rules)], lambda x: 1, rule_name('OTHER'))]

#-------

def prot_id(s):
    a = re.search(r"\d", s)
    b = re.search(r"_", s)
    if not a:
        return 0    
    if b:
        return int(s[a.start():b.start():]) 
    return int(s[a.start()::]) 

def prot_name(s):
    a = re.search(r"\d", s)
    m = 0 if not a else a.start()
    return s[0:m:] if m else s

def prot_nums(s):
    s.replace('?', '1')
    m = []
    if s.find('_') == -1:
        return set()
    
    t = s[s.find('_') + 1::]
    for i in range(len(t)):
        if t[i] == '_' or t[i] == '-':
            for j in range(int(t[i - 1])+1, int(t[i + 1])):
                m.append(j)
        elif t[i].isdigit():
            m.append(int(t[i]))
    return set(m)


def prot(protocols):
    prot_splited = []
    for p in protocols:
        if len(prot_nums(p)) == 0:
            prot_splited.append([prot_name(p), prot_id(p), ''])
        for i in prot_nums(p):
            prot_splited.append([prot_name(p), prot_id(p), str(i)])    
    prot_splited = [list(x) for x in set(tuple(x) for x in prot_splited)]
    return [ParsingRule.match_reg(re.compile(f'{p[0]}{p[1]}_(([0-9]*{p[2]})|{p[2]})[0-9]*') , lambda x: 1, f'{p[0]}{p[1]}_0{p[2]}') for p in prot_splited]


Задаем правила парсинга:

In [45]:
lambda_bin = lambda x: 1

# 'ABORT':
RemarksABORT = [ParsingRule.match_in(['ЖТ', 'ЖТ,'], lambda_bin, 'ЖТ'),
ParsingRule.match_in(['ДС', 'ДЛ СХЕМА', 'ДЛ СХ'], lambda_bin, 'СХ'),
ParsingRule.match_in(['БЖТ', 'БЕЗ ЖТ'], lambda_bin, 'БЖТ'),
ParsingRule.match_str('ФК', lambda_bin, 'ФК'), 
ParsingRule.match_str('ЛК', lambda_bin, 'ЛК')]

# 'HEALTH':
RemarksHEALTH = [ParsingRule.match_in(['МАТСТИТ', 'МАСТИТ,', 'МАСТИТ3', 'МАСТИТ'], lambda_bin, 'МАСТИТ'),
ParsingRule.match_in(['ХРОМОТА', 'ХРОМАТА'], lambda_bin, 'ХРОМОТА')]

#'BROKE':
RemarksBROKE = [ParsingRule.match_in(['НОГИ', 'БРАКНОГИ'], lambda_bin, 'НОГИ'),
ParsingRule.match_in(['2СОСКА', '2 СОСКА', 'ТУГОДОЙ', 'ATP4', 'АТР4', 'АТР1',  'АТР2', 'АТР3', 'ВЫМЯ'], lambda_bin,'ВЫМЯ'),
ParsingRule.match_in(['БЕЛЬМО', 'СЛЕПАЯ'], lambda_bin, 'СЛЕПАЯ')]

 #'OPEN':
RemarksOPEN = [ParsingRule.match_in(['ФК', 'ФК, СТЕЛ', 'ФОЛ', 'ФЛ'], lambda_bin, 'ФК'),
ParsingRule.match_in(['ЛК', 'ЛКЭ'], lambda_bin, 'ЛК'),
ParsingRule.match_in(['ЖТ', 'ЖТ Э', 'ЖТ,', 'ЖТ3',  'ЖТМ', 'ЮЖТ', 'ЖТ,ТОЩАЯ','ЖТ ПОЛИК', 'ЖТ ДЛ СХ'], lambda_bin,'ЖТ'),
ParsingRule.match_in(['БЕЗЖТ', 'БЕЗ ЖТ'], lambda_bin,'БЖТ'),
ParsingRule.match_in(['АБС', 'БРАК АБС', 'БР МАТКА', 'БРАКАБС', 'АБ МАТКИ','ЖТ/АБС'], lambda_bin, 'АБЦ'),
ParsingRule.match_in(['ПК', 'ПОЛИКИСТ', 'ПОЛИКИС', 'ЖТ ПОЛИК'], lambda_bin,'ПК'),
ParsingRule.match_in(['ГПФ', 'ГИПОФУНК', 'ГИПОФ'], lambda_bin, 'ГПФ'),
ParsingRule.match_in(['БПАК', 'БРАК', 'ХУДАЯ', 'БРАКНОГИ', 'БРАКТОЩА', 'БРАКЖИРН', 'БР ЖИРНА', 'ИЗ БРАКА'], lambda_bin, 'БРАК'),
ParsingRule.match_in(['ДЛ СХ', 'ДЛСХ', 'ДЛ СХЕМА', 'ДЛСХ ОХО','11 СХ','ЖТ ДЛ СХ'], lambda_bin, 'CХ')]
        
#'DNB':
RemarksDNB = [ParsingRule.match_in(['ГИНЕКОЛ' , 'ГЕНИКОЛ' , 'ГЕНЕКОЛ'], lambda_bin, 'ГИН'),
ParsingRule.match_in(['АТП' , 'АТ'], lambda_bin, 'АТ'),
ParsingRule.match_in(['БРАК МОЛ', 'МОЛОКО'], lambda_bin, 'МОЛ'),
ParsingRule.match_in(['ЖИРНАЯ', 'ЖИРН',  'ВЕС', 'ХУДАЯ', 'РОСТ', 'НЕДОРОСТ', 'РАЗВИТИЕ' , 'УЗКИЙТАЗ', 'МЕЛКАЯ'], lambda_bin, 'ВЕС'),
ParsingRule.match_in(['АБС', 'АБСЦЕСС' , 'БРАК АБС' , 'ABC' , 'АБЦ' , 'АБЦЕС' , 'АБЦ ОПУХ' , 'АБСМАТКА', 'БЕЗМАТКИ' , 'БЕЗМЕТКИ'], lambda_bin, 'АБЦ'),
ParsingRule.match_in(['АГАЛАКТ', 'АГАЛАКТИ'], lambda_bin, 'АГ'),
ParsingRule.match_in(['МАСТИТ', 'МАСТ'], lambda_bin, 'МАСТ'),
ParsingRule.match_in(['ТУГОДОЙ', 'НАДОЙ', 'ТОНК СОС',  'ВЫМЯТРОЦ' ], lambda_bin, 'ВЫМЯ'),
ParsingRule.match_in(['БРАКНОГИ', 'ТРОЦНОГИ', 'НОГИТРОЦ', 'НОГИ ТОЩ'], lambda_bin,'НОГИ'),
ParsingRule.match_in(['ТОЩ', 'ТОЩ СУСТ', 'ТОЩЬ','НОГИ ТОЩ'], lambda_bin,'ТОЩ'),
ParsingRule.match_in(['СЛЕПАЯ', 'ГЛАЗА', 'БЕЗГЛАЗА'], lambda_bin, 'CЛЕПАЯ')]
               
#'DRY':

RemarksDRY = [ParsingRule.match_in(['CEBA', 'СЕВА', 'СEBA', 'АТР124', 'AZIT'], lambda_bin, 'СЕВА')]

In [46]:
RuleBin  = ParsingRule(lambda x: 1, lambda x: 1, 'BIN')
RuleNum  = ParsingRule.match_reg(re.compile('[0-9]*'), lambda x: int(x) if x.isdigit() else 0, 'INT') 
RuleDays = ParsingRule.match_reg(re.compile('[0-9]* ((ДНИ)|(DAYS))'), lambda x: int(x[0:-4]) if re.match('[0-9]* (ДНИ)', x) else int(x[0:-5]), 'DAYS')

EventRules = {
    'ABORT'   : comp([RuleDays.copy()] + RemarksABORT, remarks('ABORT')),
    'TOSCM'   : uniq(remarks('TOSCM')), 
    'NULSCM'  : [RuleBin.copy()],
    'DNB'     : comp(RemarksDNB, remarks('DNB')),
    'BRED'    : uniq(remarks('BRED')), 
    'FRESH'   : [RuleBin.copy()],  
    'PREG'    : [RuleDays.copy()], 
    'PREGBEF' : [RuleDays.copy()],
    'DRY'     : comp(RemarksDRY, ('DRY')),
    'DRY2'    : [RuleBin.copy()],
    'OPEN'    : comp(RemarksOPEN,('OPEN')), 
    #-------
    'WEIGHT'  : [RuleNum.copy()],
    'DEAD'    : uniq(remarks('DEAD')),
    'MOVE'    : [RuleBin.copy()],
    'SOLD'    : uniq(remarks('SOLD')),
    #-------
    'VAC'     : uniq(remarks('VAC')),
    'VACVIR'  : uniq(remarks('VACVIR')),
    'FOOTRIM' : uniq(remarks('FOOTRIM')),
    'HEALTH'  : comp(RemarksHEALTH, remarks('HEALTH')),
    'BROKE'   : comp(RemarksBROKE, remarks('BROKE')),
    'POT'     : uniq(remarks('POT')),
    'ILLMISC' : prot(remarks('ILLMISC')),
    'LAME'    : prot(remarks('LAME')),
    'KETOS'   : uniq(remarks('KETOS')),
    'MAST'    : prot(remarks('MAST')),
    'METR'    : uniq(remarks('METR')),
    'PARES'   : uniq(remarks('PARES')),
    'RP'      : uniq(remarks('RP')),
}

#Get All Rules:
print("Правила парсинга:")
for event in EventRules.keys():
    print(event + ':')
    for rule in EventRules[event]:
        rule.name = event + '_' + rule_name(rule.name.upper())
        print(rule.name)
    print('--------------------')

Правила парсинга:
ABORT:
ABORT_DAYS
ABORT_JT
ABORT_SH
ABORT_BJT
ABORT_FK
ABORT_LK
ABORT_OTHER
--------------------
TOSCM:
--------------------
NULSCM:
NULSCM_BIN
--------------------
DNB:
DNB_GIN
DNB_AT
DNB_MOL
DNB_VES
DNB_ABZ
DNB_AG
DNB_MAST
DNB_VYMA
DNB_NOGI
DNB_TOS
DNB_CLEPAA
DNB_OTHER
--------------------
BRED:
BRED_1H11917
BRED_1H11673
BRED_1H11978
BRED_CALGARY
BRED_1H11344
BRED_777H10778
BRED_PLUTON
BRED_777H10771
BRED_777H10777
BRED_777H10574
BRED_777H10690
BRED_1H13162
BRED_200H528
--------------------
FRESH:
FRESH_BIN
--------------------
PREG:
PREG_DAYS
--------------------
PREGBEF:
PREGBEF_DAYS
--------------------
DRY:
DRY_SEVA
DRY_OTHER
--------------------
DRY2:
DRY2_BIN
--------------------
OPEN:
OPEN_FK
OPEN_LK
OPEN_JT
OPEN_BJT
OPEN_ABZ
OPEN_PK
OPEN_GPF
OPEN_BRAK
OPEN_CH
OPEN_OTHER
--------------------
WEIGHT:
WEIGHT_INT
--------------------
DEAD:
DEAD_PROCEE
DEAD_PNEVMONIA
DEAD_TRAVMY
DEAD_POSLERODOVOE
--------------------
MOVE:
MOVE_BIN
--------------------
SOLD:
SOLD

In [47]:
for rule in EventRules['MAST']:
    print(rule.name)
    if rule.match('СКМ4_34'):
        print('#')

MAST_KM1_03
MAST_SKM2_01
MAST_TM1_01
MAST_TM1_04
MAST_KM3_03
MAST_SKM4_01
MAST_TM5_03
MAST_TM3_04
MAST_SKM4_04
#
MAST_KM2_02
MAST_KM5_03
MAST_SKM6_03
MAST_SKM2_04
MAST_SKM5_02
MAST_KM3_01
MAST_KM1_01
MAST_SKM6_01
MAST_TM5_01
MAST_KM1_04
MAST_KM5_01
MAST_KM4_02
MAST_TM5_04
MAST_KM5_04
MAST_KM3_04
MAST_KM6_02
MAST_SKM6_04
MAST_SKM3_03
MAST_KM2_03
MAST_TM6_02
MAST_TM3_02
MAST_SKM3_01
MAST_TM4_03
MAST_TM1_02
MAST_KM2_01
MAST_SKM3_04
MAST_KM4_03
MAST_MV0_0
MAST_KM2_04
MAST_TM2_04
MAST_SKM2_02
MAST_KM6_03
MAST_SKM5_01
MAST_MB0_0
MAST_KM1_02
MAST_TM4_01
MAST_TM6_03
MAST_TM4_04
MAST_KM4_01
MAST_KM3_02
MAST_TM5_02
MAST_SKM6_02
MAST_KM6_01
MAST_KM5_02
MAST_KM6_04
MAST_KM4_04
MAST_TM4_02
MAST_TM1_03
MAST_TM3_03
MAST_SKM4_03
#
MAST_TM6_04


In [48]:
for event in EventRules.keys():
    for rule in EventRules[event]:
        #print(rule.name)
        df[rule.name] = df['remark'].apply(lambda x: rule.parse(x) if rule.match(x) else 0)
        df[rule.name] = df[rule.name] * (df['event'] == event)
    

In [51]:
cond = (df['event'] == 'MAST')  & (df['MAST_TM4_01'] != 0)
df[cond].head()



,event_id,id,sex,lactation_number,birth_result,birth_difficult,birthday,days_no_milking,days_pregnant,group_id,...,MAST_KM4_04,MAST_TM4_02,MAST_TM1_03,MAST_TM3_03,MAST_SKM4_03,MAST_TM6_04,METR_E1,PARES_PAREZ,RP_POSLED2,RP_POSLED
21742,21742,400377,F,3,FA,1,18.07.2014,51,213,3,...,0,1,0,0,0,0,0,0,0,0


In [50]:
print(df[cond]['remark'].tolist())

['ТМ4_124']


In [27]:
df.to_csv('counters.csv')

In [ ]:
df[df['MAST_TM7_00'] == 0].head()

## Генерация оставшихся каунтеров

### Пол и дети

In [ ]:
df['sex'] = df['sex'].apply(lambda x: 1 if x =='F' else 0)

In [ ]:
df['childs_fa'] = df['birth_result'].apply(lambda x: str(x).count('FA'))
df['childs_ma'] = df['birth_result'].apply(lambda x: str(x).count('MA'))
df['childs_fd'] = df['birth_result'].apply(lambda x: str(x).count('FD'))
df['childs_md'] = df['birth_result'].apply(lambda x: str(x).count('MD'))

df['childs_m'] = df['birth_result'].apply(lambda x: str(x).count('M'))
df['childs_f'] = df['birth_result'].apply(lambda x: str(x).count('F'))
df['childs_d'] = df['birth_result'].apply(lambda x: str(x).count('D'))
df['childs_a'] = df['birth_result'].apply(lambda x: str(x).count('A'))


In [ ]:
df=df.drop(columns=['birth_result'])
df.dtypes

**TODO** check

### Дата рождения

In [ ]:
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df['birthday']  = pd.to_datetime(df['birthday'], infer_datetime_format=True)
df['age']  = ((df['date'] - df['birthday']) / np.timedelta64(1, 'D')).astype(int)

In [ ]:
df=df.drop(columns=['birthday'])
df.dtypes

### Дата события

In [ ]:
month=['jan','feb','mar','apr','may','jun', 'jul', 'aug', 'sep', 'oct', 'nov',' dec']
for m in range (12):
    df[month[m]]=df['date'].apply(lambda x: 1 if x.month == m + 1 else 0)
df.head()

In [ ]:
STATUS_TYPES = ['FRESH', 'TOSCM', 'NULSCM', 'BRED', 'OPEN', 'ABORT', 'PREG', 'PREGBEF', 'DRY', 'DRY2', 'FOOTRIM', 'DNB', 'DEAD','SOLD','BROKE']
class StatusType(IntEnum):
    #------ Осемение
    FRESH   = 1110  # Отёл
    TOSCM   = 1121  # На схему гормональной стимуляции
    NULSCM  = 1121  # Со схемы гормональной стимуляции
    BRED    = 1130  # Осеменение
    #------ Не успех
    OPEN    = 1211  # Яловая
    ABORT   = 1212  # Аборт
    #------ Успех
    PREG    = 1311  # Стельность
    PREGBEF = 1312  # Стельность с выявлением более раннего оплодотворения
    DRY     = 1321  # Сухостой (фаза 1)
    DRY2    = 1322  # Сухостой (фаза 2)
    FOOTRIM = 1330  # Обрезка копыт
    #----- Final status
    DNB     = 1411  # Выбраковка  
    DEAD    = 1412  # Смерть
    SOLD    = 1413  # Продажа
    BROKE   = 1414  # Деффект
    
    @staicmethod
    def match(label):
        global STATUS_TYPES
        return label in STATUS_TYPES


ILLNESS_TYPES = ['MAST', 'LAME', 'KETOS', 'METR', 'PARES', 'ILLMISC', 'HEALTH', 'POT', 'VAC', 'VACVIR']
class IllnessType(IntEnum):
    #----- Основные болезни
    MAST     = 2010 #
    LAME     = 2020 #
    KETOS    = 2030 #
    METR     = 2040 #
    PARES    = 2050 #
    RP       = 2060 #
    
    #------ Другие болезни    
    ILL_BUR  = 2100 # бурсит
    ILL_ABC  = 2110 # абцесс
    ILL_RUB  = 2120 # рубец
    ILL_RIN  = 2130 # рин
    ILL_BLD  = 2140 # ран
    ILL_DMG  = 2150 # повреждение ('СУСТАВ', 'УШИБ',  'ПУТЫ')
    ILL_GAST = 2160 # гастро
    ILL_PNEV = 2170 # пневмно
    ILL_LEAN = 2180 # худоба
    ILL_DIAR = 2190 # диарея 
    ILL_MICS = 2200 # остальное 'ТУГОДОЙ', 'ГИНТЕЛ', 'ТИМПАНИЯ', 'КОЛИ', 'РОТА'

    #------ События лечения
    HEALTH   = 2300 #
    POT      = 2310 #
    VAC      = 2320 #
    VACVIR   = 2330 #
    
    @staicmethod
    def match(label):
        global ILLNESS_TYPES
        return label in ILLNESS_TYPES

    
ACTION_TYPES = ['WEIGHT', 'MOVE']
class ActionType(IntEnum):
    WEIGHT   = 3310 #
    MOVE     = 3320 #
    
    @staicmethod
    def match(label, remark):
        global ACTION_TYPES
        return label in ACTION_TYPES
        
    
class EventType(IntEnum):
    STATUS  = 1 
    ILL     = 2 
    ACTION  = 3
    
    @staticmethod
    def from_str(label):
        global EventName
        event = 
        if StatusType.match(event):
            return EventType.STATUS
        elif IllnesType.match(event):
            return EventType.ILL
        elif ActionType.match(event):
            return EventType.ACTION
        return EventType(-1)


## Учёт истории и переводы
Теперь мы переходим к обработке контексто-зависимых забытий. Для этого надо иметь понимаение какое событие произошло ДО текущего события и ПОСЛЕ текущего события.

Как понять очередность событий, если они происходят в один день? Для начала надо посмотреть на примеры такого.

In [ ]:
sf = df.groupby(['id', 'date']).count()
sf[sf['event'] > 3]

Иногда (но очень редко), с коровой происходит несколько событий в один день. Как правило эти события отделены переводом, а между переводами порядок не важен.

Кстати параметр "Предыдущая группа животного" указывает на группу в предыдущем событии! Сортировка событий в течении дня происходит так:
1. Разбить события на на группы (каждая группа отвечает что делали с животным когда оно было в соотвествующей группе)
2. Упорядочить группы в соответсвии с информацией о перводах
3. Упорядочить события внутри группы по порядку из Enuma

### События перемещения:
Надо проверить, что события перемещения отображают корректную информацию

In [ ]:
df['incorrect_move'] = df.apply(lambda row: 1 if row.event == EventType.MOVE and int(row.remark[1:4]) != row.group_id else 0, axis=1)
df[df['incorrect_move'] == 1].head()

### Сортировка событий:

**TODO**:
Написано максимально криво надо подправить. К тому же есть вопросы к эффективности

In [ ]:
print(df[df['event'] == EventType.FRESH]['group_id'].unique().tolist())

cond = ((df['event'] == EventType.FRESH) & (df['group_id'] == 18))
df[cond].head(10)

In [ ]:
import itertools
def path_sort(edge_list):
    m = len(edge_list)
    if m == 1:
        return edge_list
    
    for sigma in itertools.permutations(list(range(m))):
        bad=False
        for i in range(1, m):
            if edge_list[sigma[i]][1] != edge_list[sigma[i-1]][0]:
                bad=True
                break
        if not bad:
            edge_list_fixed = [edge_list[sigma[i]] for i in range(1, m)]
            return edge_list_fixed
    return edge_list


def get_day_ordering(r):
    
    cow_id   = r.id
    ev_date  = r.date 
    ev_index = r.event_index
    
    global df
    today = df[(df['id'] == cow_id) & (df['date'] == ev_date)].copy()
    
    moves = today[today['event'] == EventType.MOVE]
    if moves.shape[0] == 0:
        today.sort_values(by=['event'])
        today['ord'] = list(range(1, today.shape[0] + 1, 1))
        return (today[today['event_index'] == ev_index]['ord'].iloc[0])
   
    moves['to']   = moves['remark'].apply(lambda x: int(x[5::]))
    moves['from'] = moves['remark'].apply(lambda x: int(x[1:4:]))
    
    edge_list  = path_sort(moves.apply(lambda row: (row['from'], row['to']), axis=1).tolist())
    path = [edge[0] for edge in edge_list]
    path.append(edge_list[-1][1])
    
    try:
        today['group_ord'] = today['group_id'].apply(lambda x: path.index(x))
        today.sort_values(by=['group_ord', 'event'])
    except:
        #print('------------------')
        #print(edge_list)
        #print(row)
        #print('------------------')
        today.sort_values(by=['event'])
        
    today['ord'] = list(range(1, today.shape[0] + 1, 1))
    
    return (today[today['event_index'] == ev_index]['ord'].iloc[0])

In [ ]:
df['ord'] = df.apply(lambda row: get_day_ordering(row), axis=1)
df.sort_values(by=['id', 'date', 'ord'])

**TODO** Спросить что вообще с номерами групп происходит:

In [ ]:
df[((df['date'] == '7/29/2022') & (df['id'] == 8036))].head(10)

In [ ]:
df[((df['date'] == '7/22/2022') & (df['id'] == 807163))].head(10)

На самом деле такое решение имеет смысл только если нет ситуации когда животное кидают из группы в группу по циклу (например перевели из 3 в 12 потом обратно в 3), в таком случае восстановить порядок наверняка невозможно. Но судя по всему такого не происходит

###  Cобытия выздоровления и болезней:
Заболевания КЕТОЗ, МЕТРИТ, ПАРЕЗ, ПОСЛЕД, ПРОФОТ протекают в новотельный период и имеют общий критерий выписки: все послеотельные осложнения животного вылечили, корова пригодна к дальнейшей работе и воспроизводству (примечание при выписке ГИНЕКОЛ). Примечание МОЛОДНЯК к событию ЗДОРОВА имеют только животные 0-й лактации (телки и бычки). Примечание ПРОЧИЕ к событию ЗДОРОВА имеют животные, перенесшие прочие заболевания (событие БОЛЕЗНЬ).

In [ ]:
df.head()

# Генерация новых фичей